In [1]:
# OpenWeatherMap API credentials
api_key = "00e8e808c9ee63e1a6848d838daabe6d"

In [2]:
import requests
import pandas as pd

# Load your dataset (replace 'your_dataset.csv' with your actual file)
df = pd.read_csv('Austin_Final_2022-06-18.csv')

In [4]:
    # Function to get all air pollution components from OpenWeatherMap for given lat and lon
def get_air_pollution_data(lat, lon):
        url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}"
        
        # Make the API request
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()  # Parse the response to JSON
            main_aqi = data['list'][0]['main']['aqi']  # AQI value
            
            components = data['list'][0]['components']  # All pollutant components
            co2 = components.get('co2', None)
            co = components.get('co', None)       # Carbon Monoxide
            no = components.get('no', None)       # Nitric Oxide
            no2 = components.get('no2', None)     # Nitrogen Dioxide
            o3 = components.get('o3', None)       # Ozone
            so2 = components.get('so2', None)     # Sulfur Dioxide
            pm25 = components.get('pm2_5', None)  # Fine Particulate Matter (PM2.5)
            pm10 = components.get('pm10', None)   # Coarse Particulate Matter (PM10)
            nh3 = components.get('nh3', None)     # Ammonia

            return main_aqi, co2, co, no, no2, o3, so2, pm25, pm10, nh3
        
        else:
            return [None] * 9  # Return None for all fields if the request fails

    # Applying the function to each row in the dataset
    df[['AQI', 'CO2', 'CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10', 'NH3']] = pd.DataFrame(
        df.apply(lambda row: get_air_pollution_data(row['latitude_coordinate'], row['longitude_coordinate']), axis=1).tolist()
    )

    # units of all data is µg/m³

KeyboardInterrupt: 

In [4]:
# Save the updated DataFrame to a new CSV file
df.to_csv('dataset_with_aqi.csv', index=False)

print("AQI data added successfully!")

AQI data added successfully!
